# Ajuste da versão especializada do modelo de linguagem BERTimbau em uma tarefa de classificação de tokens (NER) com o dataset LeNER-Br

- **Credit**: this notebook is copied/pasted with small changes from [PyTorch Examples](https://huggingface.co/docs/transformers/notebooks#pytorch-examples) of Hugging Face (notebook [token_classification.ipynb](https://github.com/huggingface/notebooks/blob/master/examples/token_classification.ipynb)).
- **Author**: [Pierre GUILLOU](https://www.linkedin.com/in/pierreguillou/)
- **Date**: 12/20/2021
- **Blog post**: [NLP | Modelos e Web App para Reconhecimento de Entidade Nomeada (NER) no domínio jurídico brasileiro](https://medium.com/@pierre_guillou/nlp-modelos-e-web-app-para-reconhecimento-de-entidade-nomeada-ner-no-dom%C3%ADnio-jur%C3%ADdico-b658db55edfb)

## Visão geral

Neste notebook, veremos como ajustar um dos modelos [🤗 Transformers](https://github.com/huggingface/transformers) para uma tarefa de classificação de token, que é a tarefa de prever um rótulo para cada token .

![Widget inference representing the NER task](https://github.com/huggingface/notebooks/raw/8044bbce25bed20a79e5488040a41d3c32575cec/examples/images/token_classification.png)

As tarefas de classificação de token mais comuns são:

- NER (reconhecimento de entidade nomeada) Classifica as entidades no texto (pessoa, organização, localização...).
- POS (marcação gramatical) Classifique gramaticalmente os tokens (substantivo, verbo, adjetivo...)
- Chunk (Chunking) Classificar gramaticalmente os tokens e agrupá-los em “pedaços” que vão juntos

Veremos como carregar facilmente um conjunto de dados para esses tipos de tarefas e usar a API `Trainer` para ajustar um modelo nele.

Este notebook foi desenvolvido para ser executado em qualquer tarefa de classificação de token, com qualquer ponto de verificação de modelo do [Model Hub](https://huggingface.co/models), desde que esse modelo tenha uma versão com um cabeçote de classificação de token e um tokenizer rápido (verifique [esta tabela](https://huggingface.co/transformers/index.html#bigtable) se for esse o caso). Poderão ser necessários apenas alguns pequenos ajustes se você decidir usar um conjunto de dados diferente daquele usado aqui. Dependendo do modelo e da GPU que você está usando, pode ser necessário ajustar o tamanho do lote para evitar erros de falta de memória. Defina esses três parâmetros e o resto do notebook deverá funcionar sem problemas:

## Configuração

In [ ]:
task = "ner" # Should be one of "ner", "pos" or "chunk"

# model_checkpoint = "neuralmind/bert-base-portuguese-cased"
# model_checkpoint = "neuralmind/bert-large-portuguese-cased"
model_checkpoint = "pierreguillou/bert-base-cased-pt-lenerbr"

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Se você estiver abrindo este Notebook no colab, provavelmente precisará instalar 🤗 Transformers e 🤗 Datasets. Remova o comentário da célula a seguir e execute-a.

In [ ]:
!pip install datasets seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 28.2 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=d52c675caa358b4cb4cf2a31aec28e7d505ab5600b1e10b3650669302f551114
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
!pip install -U accelerate
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.7 MB/s eta 0:00:00


Se você estiver abrindo este notebook localmente, certifique-se de que seu ambiente tenha uma instalação da última versão dessas bibliotecas.

Para poder compartilhar seu modelo com a comunidade e gerar resultados como o mostrado na imagem abaixo por meio da API de inferência, há mais alguns passos a seguir.

Primeiro você deve armazenar seu token de autenticação do site Hugging Face (inscreva-se [aqui](https://huggingface.co/join) se ainda não o fez!), em seguida, execute a seguinte célula e insira seu nome de usuário e senha:

Então você precisa instalar o Git-LFS. Remova o comentário das seguintes instruções:

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


Certifique-se de que sua versão do Transformers seja pelo menos 4.11.0, já que a funcionalidade foi introduzida nessa versão:

In [ ]:
import transformers

print(transformers.__version__)

4.33.2


In [ ]:
import datasets

print(datasets.__version__)

2.14.5


In [ ]:
import pathlib
from pathlib import Path

import pandas as pd

In [ ]:
from datasets import Dataset, DatasetDict

Você pode encontrar uma versão de script deste notebook para ajustar seu modelo de forma distribuída usando várias GPUs ou TPUs [aqui](https://github.com/huggingface/transformers/tree/master/examples/token-classification) .

## Carregando o conjunto de dados

Usaremos a biblioteca [🤗 Datasets](https://github.com/huggingface/datasets) para baixar os dados e obter a métrica que precisamos usar para avaliação (para comparar nosso modelo com o benchmark). Isso pode ser feito facilmente com as funções `load_dataset` e `load_metric`.

In [ ]:
from datasets import load_dataset, load_metric

Para nosso exemplo aqui, usaremos o [conjunto de dados LeNER-Br](https://huggingface.co/datasets/lener_br). O notebook deve funcionar com qualquer conjunto de dados de classificação de token fornecido pela biblioteca 🤗 Datasets. Se você estiver usando seu próprio conjunto de dados definido a partir de um arquivo JSON ou CSV (consulte a [documentação de conjuntos de dados](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) para saber como carregá-los ), poderá ser necessário alguns ajustes nos nomes das colunas utilizadas.

In [ ]:
datasets = load_dataset("lener_br")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/7828 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1177 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1390 [00:00<?, ? examples/s]

O objeto `datasets` em si é [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), que contém uma chave para o conjunto de treinamento, validação e teste.

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 7828
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1177
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1390
    })
})

Podemos ver que todos os conjuntos de treinamento, validação e teste têm uma coluna para os tokens (os textos de entrada divididos em palavras) e uma coluna de rótulos para cada tipo de tarefa que introduzimos anteriormente.

Para acessar um elemento real, você precisa primeiro selecionar uma divisão e depois fornecer um índice:

In [ ]:
datasets["train"][0]

{'id': '0',
 'tokens': ['EMENTA',
  ':',
  'APELAÇÃO',
  'CÍVEL',
  '-',
  'AÇÃO',
  'DE',
  'INDENIZAÇÃO',
  'POR',
  'DANOS',
  'MORAIS',
  '-',
  'PRELIMINAR',
  '-',
  'ARGUIDA',
  'PELO',
  'MINISTÉRIO',
  'PÚBLICO',
  'EM',
  'GRAU',
  'RECURSAL',
  '-',
  'NULIDADE',
  '-',
  'AUSÊNCIA',
  'DE',
  'INTERVENÇÃO',
  'DO',
  'PARQUET',
  'NA',
  'INSTÂNCIA',
  'A',
  'QUO',
  '-',
  'PRESENÇA',
  'DE',
  'INCAPAZ',
  '-',
  'PREJUÍZO',
  'EXISTENTE',
  '-',
  'PRELIMINAR',
  'ACOLHIDA',
  '-',
  'NULIDADE',
  'RECONHECIDA',
  '.'],
 'ner_tags': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

The labels are already coded as integer ids to be easily usable by our model, but the correspondence with the actual categories is stored in the `features` of the dataset:

In [ ]:
datasets["train"].features[f"ner_tags"]

Sequence(feature=ClassLabel(names=['O', 'B-ORGANIZACAO', 'I-ORGANIZACAO', 'B-PESSOA', 'I-PESSOA', 'B-TEMPO', 'I-TEMPO', 'B-LOCAL', 'I-LOCAL', 'B-LEGISLACAO', 'I-LEGISLACAO', 'B-JURISPRUDENCIA', 'I-JURISPRUDENCIA'], id=None), length=-1, id=None)

Portanto, para as tags NER, 0 corresponde a 'O', 1 a 'B-PER' etc... Além de 'O' (o que significa nenhuma entidade especial), existem quatro rótulos para NER aqui, cada um prefixado com 'B-' (para iniciante) ou 'I-' (para intermediário), que indica se o token é o primeiro do grupo atual com o rótulo ou não:
- 'PESSOA' for person
- 'ORGANIZACAO' for organization
- 'LOCAL' for location
- ....

Como os rótulos são listas de `ClassLabel`, os nomes reais dos rótulos estão aninhados no atributo `feature` do objeto acima:

In [ ]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['O',
 'B-ORGANIZACAO',
 'I-ORGANIZACAO',
 'B-PESSOA',
 'I-PESSOA',
 'B-TEMPO',
 'I-TEMPO',
 'B-LOCAL',
 'I-LOCAL',
 'B-LEGISLACAO',
 'I-LEGISLACAO',
 'B-JURISPRUDENCIA',
 'I-JURISPRUDENCIA']

Para ter uma ideia da aparência dos dados, a função a seguir mostrará alguns exemplos escolhidos aleatoriamente no conjunto de dados (decodificando automaticamente os rótulos de passagem).

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,id,tokens,ner_tags
0,3462,"[7, -, Não, ofende, o, art, ., 59, do, CP, a, fixação, da, pena-base, acima, do, mínimo, legal, se, as, circunstâncias, judiciais, desfavoráveis, resultaram, da, análise, das, condições, pessoais, do, recorrente, ,, como, sua, conduta, social, e, personalidade, ,, bem, como, das, circunstâncias, e, consequências, do, delito, ,, que, evidenciaram, sua, alta, culpabilidade, e, a, maior, necessidade, de, reprovação, e, prevenção, do, crime, ,, não, prosperando, a, alegação, de, utilização, ,, na, sentença, condenatória, ,, de, elementos, constitutivos, do, próprio, tipo, penal, .]","[O, O, O, O, O, B-LEGISLACAO, I-LEGISLACAO, I-LEGISLACAO, I-LEGISLACAO, I-LEGISLACAO, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
1,7264,"[13, .]","[O, O]"
2,4519,"[Decisão, :, O, Tribunal, ,, por, unanimidade, ,, rejeitou, os, embargos, de, declaração, ,, nos, termos, do, voto, do, relator, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
3,2271,"[Na, aferição, da, legitimidade, passiva, deve-se, tomar, por, base, o, direito, abstratamente, invocado, e, a, pertinência, subjetiva, entre, o, pedido, e, as, partes, chamadas, em, juízo, ,, analisada, conforme, a, Teoria, da, Asserção, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
4,5500,"[Habeas, corpus, não, conhecido, .]","[O, O, O, O, O]"
5,6213,"[Conforme, esclarecido, pelo, Relator, daquele, processo, administrativo, ,, o, i, ., Min, ., Aldir, Passarinho, Junior, ,, “, as, decisões, dos, TRF, 's, que, impliquem, aumento, de, despesa, ,, para, que, tenham, eficácia, ,, devem, ser, submetidas, à, homologação, do, Colegiado, do, CJF, ,, a, teor, da, exigência, do, art, ., 5º, ,, IV, ,, da, Lei, nº, 8.472, ,, de, 14, de, outubro, de, 1992, ,, e, do, art, ., 4º, ,, IV, ,, do, Regimento, Interno, do, CJF, ”, (, fl, ., 220, ), .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-PESSOA, I-PESSOA, I-PESSOA, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ORGANIZACAO, O, O, O, O, O, O, B-LEGISLACAO, I-LEGISLACAO, I-LEGISLACAO, I-LEGISLACAO, I-LEGISLACAO, I-LEGISLACAO, I-LEGISLACAO, I-LEGISLACAO, I-LEGISLACAO, I-LEGISLACAO, O, O, B-TEMPO, I-TEMPO, I-TEMPO, I-TEMPO, I-TEMPO, O, O, O, B-LEGISLACAO, I-LEGISLACAO, I-LEGISLACAO, I-LEGISLACAO, I-LEGISLACAO, I-LEGISLACAO, I-LEGISLACAO, I-LEGISLACAO, I-LEGISLACAO, I-LEGISLACAO, I-LEGISLACAO, O, O, O, O, O, O, O]"
6,3026,"[O, Estado, quer, ,, além, da, punibilidade, ,, a, '', utilidade, '', da, persecução, penal, acoplada, à, efetividade, administrativa, ,, ou, seja, ,, poder, exigir, do, agente, ,, apto, em, inspeção, de, saúde, ,, o, cumprimento, do, restante, do, Serviço, Militar, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
7,5333,"[Porém, ,, se, a, intenção, tiver, o, escopo, de, vedar, qualquer, comentário, do, juiz, a, respeito, das, teses, levantadas, pela, defesa, ,, ignorando-as, por, completo, ,, atinge-se, a, inconstitucionalidade, ,, pois, fere, a, plenitude, de, defesa, e, o, preceito, constitucional, de, que, toda, decisão, do, Poder, Judiciário, deve, ser, fundamentada, ,, não, podendo, haver, cerceamento, por, mando, da, lei, ordinária, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
8,2751,"[Relator, :, William, de, Oliveira, Barros, ,, Julgamento, :, 10/09/2014, ,, Publicação, Dje, :, 23/09/2014, ), .]","[O, O, B-PESSOA, I-PESSOA, I-PESSOA, I-PESSOA, O, O, O, B-TEMPO, O, O, O, O, B-TEMPO, O, O]"
9,6963,"[Desnecessária, a, remessa, dos, autos, ao, Ministério, Público, do, Trabalho, ,, consoante, o, art, ., 83, ,, §, 

##Pré-processando os dados

Antes de podermos alimentar esses textos em nosso modelo, precisamos pré-processá-los. Isso é feito por um 🤗 Transformers `Tokenizer` que irá (como o nome indica) tokenizar as entradas (incluindo a conversão dos tokens em seus IDs correspondentes no vocabulário pré-treinado) e colocá-los em um formato que o modelo espera, bem como gerar o outras entradas que o modelo requer.

Para fazer tudo isso, instanciamos nosso tokenizer com o método `AutoTokenizer.from_pretrained`, que irá garantir:

- obtemos um tokenizer que corresponde à arquitetura do modelo que queremos usar,
- baixamos o vocabulário usado no pré-treinamento deste ponto de verificação específico.

Esse vocabulário será armazenado em cache, portanto não será baixado novamente na próxima vez que executarmos a célula.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

A afirmação a seguir garante que nosso tokenizer é um tokenizer rápido (apoiado por Rust) da biblioteca 🤗 Tokenizers. Esses tokenizadores rápidos estão disponíveis para quase todos os modelos e precisaremos de alguns dos recursos especiais que eles possuem para nosso pré-processamento.

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

Você pode verificar quais tipos de modelos têm um tokenizer rápido disponível e quais não têm na [grande tabela de modelos](https://huggingface.co/transformers/index.html#bigtable).

Você pode chamar esse tokenizer diretamente em uma frase:

In [ ]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [101, 15044, 22280, 117, 12230, 145, 847, 3185, 22279, 5440, 1710, 106, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Dependendo do modelo selecionado, você verá chaves diferentes no dicionário retornado pela célula acima. Eles não importam muito para o que estamos fazendo aqui (apenas saiba que são exigidos pelo modelo que instanciaremos mais tarde). Você pode aprender mais sobre eles [neste tutorial](https://huggingface.co/transformers/ preprocessing.html) se você estiver interessado.

Se, como é o caso aqui, suas entradas já foram divididas em palavras, você deve passar a lista de palavras para o seu tokenzier com o argumento `is_split_into_words=True`:

In [ ]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

{'input_ids': [101, 15044, 22280, 117, 12230, 145, 847, 3185, 22279, 5440, 1710, 139, 863, 284, 19124, 2702, 824, 22281, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Observe que os transformadores geralmente são pré-treinados com tokenizadores de subpalavras, o que significa que mesmo que suas entradas já tenham sido divididas em palavras, cada uma dessas palavras poderá ser dividida novamente pelo tokenizador. Vejamos um exemplo disso:

In [ ]:
example = datasets["train"][5]
print(example["tokens"])

['V.v', 'APELAÇÃO', 'CÍVEL', '-', 'NULIDADE', 'PROCESSUAL', '-', 'INTIMAÇÃO', 'DO', 'MINISTÉRIO', 'PÚBLICO', '-', 'INCAPAZ', 'ACOMPANHADA', 'DE', 'REPRESENTANTE', 'LEGAL', 'E', 'DE', 'ADVOGADO', '-', 'EXERCÍCIO', 'DO', 'CONTRADITÓRIO', 'E', 'DA', 'AMPLA', 'DEFESA', '-', 'AUSÊNCIA', 'DE', 'PREJUÍZOS', '-', 'VÍCIO', 'AFASTADO', '-', 'IMPROCEDÊNCIA', 'DO', 'PEDIDO', '-', 'INEXISTÊNCIA', 'DE', 'PROVA', 'QUANTO', 'AO', 'FATO', 'CONSTITUTIVO', 'DO', 'DIREITO', '.']


In [ ]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'V', '.', 'v', 'AP', '##EL', '##A', '##Ç', '##ÃO', 'C', '##Í', '##V', '##EL', '-', 'N', '##UL', '##ID', '##AD', '##E', 'PR', '##OC', '##ES', '##S', '##UA', '##L', '-', 'IN', '##TI', '##MA', '##Ç', '##ÃO', 'DO', 'M', '##IN', '##IS', '##T', '##É', '##RI', '##O', 'P', '##Ú', '##B', '##L', '##IC', '##O', '-', 'IN', '##CA', '##PA', '##Z', 'AC', '##OM', '##PA', '##N', '##HA', '##DA', 'DE', 'R', '##EP', '##RE', '##SE', '##NT', '##AN', '##TE', 'L', '##E', '##GA', '##L', 'E', 'DE', 'A', '##D', '##V', '##O', '##GA', '##DO', '-', 'E', '##X', '##ER', '##C', '##Í', '##CI', '##O', 'DO', 'CON', '##T', '##RA', '##DI', '##T', '##Ó', '##RI', '##O', 'E', 'D', '##A', 'AM', '##P', '##LA', 'DE', '##F', '##ES', '##A', '-', 'A', '##US', '##Ê', '##N', '##CI', '##A', 'DE', 'PR', '##E', '##J', '##U', '##Í', '##Z', '##OS', '-', 'V', '##Í', '##CI', '##O', 'A', '##FA', '##ST', '##AD', '##O', '-', 'I', '##MP', '##RO', '##CE', '##D', '##Ê', '##N', '##CI', '##A', 'DO', 'P', '##ED', '##ID', '##O', '-', 'IN', 

Aqui as palavras "Zwingmann" e "carne de ovelha" foram divididas em três subtokens.

Isso significa que precisamos fazer algum processamento em nossos rótulos, pois os ids de entrada retornados pelo tokenizer são maiores que as listas de rótulos que nosso conjunto de dados contém, primeiro porque alguns tokens especiais podem ser adicionados (podemos usar um `[CLS]` e um `[SEP]` acima) e por causa dessas possíveis divisões de palavras em vários tokens:

In [ ]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(50, 178)

Felizmente, o tokenizer retorna saídas que possuem um método `word_ids` que pode nos ajudar.

In [ ]:
print(tokenized_input.word_ids())

[None, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 7, 7, 7, 7, 7, 8, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 11, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 14, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 17, 18, 19, 19, 19, 19, 19, 19, 20, 21, 21, 21, 21, 21, 21, 21, 22, 23, 23, 23, 23, 23, 23, 23, 23, 24, 25, 25, 26, 26, 26, 27, 27, 27, 27, 28, 29, 29, 29, 29, 29, 29, 30, 31, 31, 31, 31, 31, 31, 31, 32, 33, 33, 33, 33, 34, 34, 34, 34, 34, 35, 36, 36, 36, 36, 36, 36, 36, 36, 36, 37, 38, 38, 38, 38, 39, 40, 40, 40, 40, 40, 40, 40, 40, 40, 41, 42, 42, 42, 43, 43, 43, 43, 44, 44, 45, 45, 46, 46, 46, 46, 46, 46, 47, 48, 48, 48, 48, 49, None]


Como podemos ver, ele retorna uma lista com o mesmo número de elementos que nossos ids de entrada processados, mapeando tokens especiais para `None` e todos os outros tokens para suas respectivas palavras. Dessa forma, podemos alinhar os rótulos com os ids de entrada processados.

In [ ]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

178 178


Aqui definimos os rótulos de todos os tokens especiais como -100 (o índice que é ignorado pelo PyTorch) e os rótulos de todos os outros tokens como o rótulo da palavra de onde eles vêm. Outra estratégia é definir o rótulo apenas no primeiro token obtido de uma determinada palavra e atribuir um rótulo de -100 aos demais subtokens da mesma palavra. Propomos aqui as duas estratégias, basta alterar o valor da seguinte flag:

In [ ]:
label_all_tokens = True

Agora estamos prontos para escrever a função que irá pré-processar nossas amostras. Nós os alimentamos no `tokenizer` com o argumento `truncation=True` (para truncar textos maiores que o tamanho máximo permitido pelo modelo) e `is_split_into_words=True` (como visto acima). Em seguida, alinhamos os rótulos com os IDs dos tokens usando a estratégia que escolhemos:

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True, max_length=512)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

Esta função funciona com um ou vários exemplos. No caso de vários exemplos, o tokenizer retornará uma lista de listas para cada chave:

In [ ]:
tokenize_and_align_labels(datasets['train'][:5])

{'input_ids': [[101, 192, 7463, 8427, 22301, 131, 12127, 9008, 22301, 22402, 16484, 187, 22360, 22339, 9008, 118, 177, 22402, 16484, 10836, 13760, 7545, 22320, 22323, 22351, 22301, 22402, 16484, 212, 8718, 250, 7665, 6072, 213, 8718, 22301, 6538, 118, 11635, 9008, 13270, 7073, 6765, 118, 11741, 22328, 22341, 6392, 22301, 212, 9008, 22317, 213, 7073, 6538, 22321, 22352, 21748, 22317, 212, 22371, 22318, 22327, 6162, 22317, 192, 22311, 278, 5650, 22341, 257, 5476, 15289, 5903, 22327, 118, 248, 18199, 6392, 11836, 22309, 118, 177, 10409, 22420, 22320, 14298, 22301, 10836, 13760, 16017, 22322, 22339, 12547, 22402, 16484, 15040, 18868, 22322, 22349, 22341, 9208, 248, 22301, 13760, 11846, 22379, 22320, 14298, 22301, 177, 5226, 22341, 22317, 118, 11635, 3341, 12547, 22402, 22301, 10836, 13760, 4529, 5869, 22351, 118, 11635, 22309, 22333, 22341, 22360, 22351, 22317, 192, 22348, 6538, 16017, 8427, 22309, 118, 11635, 9008, 13270, 7073, 6765, 11247, 7918, 22340, 6392, 22301, 118, 248, 18199, 6392,

Para aplicar esta função em todas as sentenças (ou pares de sentenças) em nosso conjunto de dados, apenas usamos o método `map` do nosso objeto `dataset` que criamos anteriormente. Isso aplicará a função em todos os elementos de todas as divisões no `dataset`, de modo que nossos dados de treinamento, validação e teste serão pré-processados ​​em um único comando.

In [ ]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/7828 [00:00<?, ? examples/s]

Map:   0%|          | 0/1177 [00:00<?, ? examples/s]

Map:   0%|          | 0/1390 [00:00<?, ? examples/s]

Melhor ainda, os resultados são automaticamente armazenados em cache pela biblioteca 🤗 Datasets para evitar perder tempo nesta etapa na próxima vez que você executar seu notebook. A biblioteca 🤗 Datasets normalmente é inteligente o suficiente para detectar quando a função que você passa para o mapa foi alterada (e, portanto, requer o não uso dos dados do cache). Por exemplo, ele detectará corretamente se você alterar a tarefa na primeira célula e executar novamente o notebook. 🤗 Datasets avisa quando usa arquivos em cache, você pode passar `load_from_cache_file=False` na chamada para `map` para não usar os arquivos em cache e forçar o pré-processamento a ser aplicado novamente.

Observe que passamos `batched=True` para codificar os textos em lotes juntos. Isso aproveita todos os benefícios do tokenizer rápido que carregamos anteriormente, que usará multithreading para tratar os textos em um lote simultaneamente.

## Ajustando o modelo

Agora que nossos dados estão prontos, podemos baixar o modelo pré-treinado e ajustá-lo. Como todas as nossas tarefas são sobre classificação de tokens, usamos a classe `AutoModelForTokenClassification`. Assim como acontece com o tokenizer, o método `from_pretrained` fará o download e armazenará em cache o modelo para nós. A única coisa que precisamos especificar é o número de rótulos para o nosso problema (que podemos obter dos recursos, como visto antes):

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at pierreguillou/bert-base-cased-pt-lenerbr and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


O aviso está nos dizendo que estamos descartando alguns pesos (as camadas `vocab_transform` e `vocab_layer_norm`) e inicializando aleatoriamente alguns outros (as camadas `pre_classifier` e `classifier`). Isso é absolutamente normal neste caso, porque estamos removendo o cabeçote usado para pré-treinar o modelo em um objetivo de modelagem de linguagem mascarada e substituindo-o por um novo cabeçote para o qual não temos pesos pré-treinados, então a biblioteca nos avisa que devemos estar bem -ajuste esse modelo antes de usá-lo para inferência, que é exatamente o que faremos.

Para instanciar um `Trainer`, precisaremos definir mais três coisas. O mais importante é o [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), que é uma classe que contém todos os atributos para customizar o treinamento. Requer um nome de pasta, que será usado para salvar os pontos de verificação do modelo, e todos os outros argumentos são opcionais:

In [ ]:
#pip install transformers[torch]

In [ ]:
#!pip install accelerate==0.20.1

In [ ]:
model_name = model_checkpoint.split("/")[-1]

# hyperparameters, which are passed into the training job

per_device_batch_size = 4
gradient_accumulation_steps = 2

#LR, wd, epochs
learning_rate = 1e-4 #2e-5 # (AdamW) we started with 3e-4, then 1e-4, then 5e-5 but the model overfits fastly
num_train_epochs = 10 # we started with 10 epochs but the model overfits fastly
weight_decay = 0.01
fp16 = True

# logs
logging_steps = 290 # melhor evaluate frequently (5000 seems too high)
logging_strategy = 'steps'
eval_steps = logging_steps

# checkpoints
evaluation_strategy = 'epoch' #steps
save_total_limit = 1 #3
save_strategy = 'epoch' #steps
save_steps = 978  #290

# best Model
load_best_model_at_end = True

# folders
model_name = model_checkpoint.split("/")[-1]
folder_model = 'e' + str(num_train_epochs) + '_lr' + str(learning_rate)

#comentado por conta do armazenamento do google drive
output_dir = '/content/drive/MyDrive/' + 'ner-lenerbr-' + str(model_name) + '/checkpoints/' + folder_model
logging_dir = '/content/drive/MyDrive/' + 'ner-lenerbr-' + str(model_name) + '/logs/' + folder_model


# get best model through a metric
metric_for_best_model = 'eval_f1'
if metric_for_best_model == 'eval_f1':
    greater_is_better = True
elif metric_for_best_model == 'eval_loss':
    greater_is_better = False

args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=learning_rate,
    per_device_train_batch_size=per_device_batch_size,
    per_device_eval_batch_size=per_device_batch_size*2,
    gradient_accumulation_steps=gradient_accumulation_steps,
    num_train_epochs=num_train_epochs,
    weight_decay=weight_decay,
    save_total_limit=save_total_limit,
    logging_steps = logging_steps,
    eval_steps = logging_steps,
    load_best_model_at_end = load_best_model_at_end,
    metric_for_best_model = metric_for_best_model,
    greater_is_better = greater_is_better,
    gradient_checkpointing = False,
    do_train = True,
    do_eval = True,
    do_predict = True,
    evaluation_strategy = evaluation_strategy,
    logging_strategy = logging_strategy,
    save_strategy = save_strategy,
    logging_dir=logging_dir,
    save_steps = save_steps,
    fp16 = fp16,
    push_to_hub=False,
)

In [ ]:
#!pip install transformers==4.28.0

Aqui definimos a avaliação a ser feita ao final de cada época, ajustamos a taxa de aprendizado, usamos o `batch_size` definido no topo do notebook e customizamos o número de épocas para treinamento, bem como a redução de peso.

O último argumento para configurar tudo para que possamos enviar o modelo para o [Hub](https://huggingface.co/models) regularmente durante o treinamento. Remova-o caso não tenha seguido os passos de instalação na parte superior do notebook. Se você quiser salvar seu modelo localmente em um nome diferente do nome do repositório que será enviado, ou se quiser enviar seu modelo para uma organização e não para seu espaço de nome, use o argumento `hub_model_id` para definir o nome do repositório (precisa ser o nome completo, incluindo seu namespace: por exemplo `"sgugger/bert-finetuned-ner"` ou `"huggingface/bert-finetuned-ner"`).

Em seguida, precisaremos de um agrupamento de dados que agrupará nossos exemplos processados ​​enquanto aplica o preenchimento para torná-los todos do mesmo tamanho (cada bloco será preenchido até o comprimento de seu exemplo mais longo). Existe um compilador de dados para esta tarefa na biblioteca Transformers, que não apenas preenche as entradas, mas também os rótulos:

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

A última coisa a definir para o nosso `Trainer` é como calcular as métricas das previsões. Aqui carregaremos a métrica [`seqeval`](https://github.com/chakki-works/seqeval) (que é comumente usada para avaliar resultados no conjunto de dados CONLL) por meio da biblioteca Datasets.

In [ ]:
metric = load_metric("seqeval")

<ipython-input-37-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


Esta métrica leva uma lista de rótulos para as previsões e referências:

In [ ]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'ORGANIZACAO': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

Portanto, precisaremos fazer um pouco de pós-processamento em nossas previsões:
- selecione o índice previsto (com o logit máximo) para cada token
- converta-o para seu rótulo de string
- ignore em todos os lugares onde definimos um rótulo de -100

A função a seguir faz todo esse pós-processamento no resultado de `Trainer.evaluate` (que é uma tupla nomeada contendo previsões e rótulos) antes de aplicar a métrica:

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Observe que descartamos a precisão/recuperação/f1 calculada para cada categoria e nos concentramos apenas na precisão/recuperação/f1/precisão geral.

Então só precisamos passar tudo isso junto com nossos conjuntos de dados para o `Trainer`:

In [ ]:
from transformers.trainer_callback import EarlyStoppingCallback

# espere early_stopping_patience x eval_steps antes de interromper o treinamento para obter um modelo melhor
early_stopping_patience = 5 #save_total_limit

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=early_stopping_patience)],
)

Agora podemos ajustar nosso modelo apenas chamando o método `train`:

In [ ]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
0,0.089000,nan,0.759842,0.826022,0.791551,0.954942
2,0.051600,nan,0.838631,0.811398,0.824790,0.969433
2,0.038300,nan,0.854522,0.841290,0.847854,0.967986
4,0.031700,nan,0.820535,0.818065,0.819298,0.962896
4,0.021900,nan,0.860711,0.874409,0.867506,0.968047
6,0.016000,nan,0.859623,0.873118,0.866318,0.965974
6,0.011500,nan,0.828669,0.885161,0.855984,0.966584
8,0.006100,nan,0.886562,0.873978,0.880225,0.970911
8,0.003200,nan,0.896081,0.890108,0.893084,0.975026
9,0.002300,nan,0.896798,0.891398,0.894090,0.975178


TrainOutput(global_step=9780, training_loss=0.031455428913570865, metrics={'train_runtime': 1678.4302, 'train_samples_per_second': 46.639, 'train_steps_per_second': 5.827, 'total_flos': 3811063508492232.0, 'train_loss': 0.031455428913570865, 'epoch': 9.99})

O método `evaluate` permite avaliar novamente no conjunto de dados de avaliação ou em outro conjunto de dados:

In [ ]:
trainer.evaluate()

{'eval_loss': nan,
 'eval_precision': 0.8967979229770662,
 'eval_recall': 0.8913978494623656,
 'eval_f1': 0.8940897325280415,
 'eval_accuracy': 0.9751778993402106,
 'eval_runtime': 8.3243,
 'eval_samples_per_second': 141.393,
 'eval_steps_per_second': 17.779,
 'epoch': 9.99}

Para obter a precisão/recall/f1 calculada para cada categoria agora que terminamos o treinamento, podemos aplicar a mesma função de antes no resultado do método `predict`:

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'JURISPRUDENCIA': {'precision': 0.8235294117647058,
  'recall': 0.745814307458143,
  'f1': 0.7827476038338658,
  'number': 657},
 'LEGISLACAO': {'precision': 0.8959854014598541,
  'recall': 0.8598949211908932,
  'f1': 0.8775692582663093,
  'number': 571},
 'LOCAL': {'precision': 0.7148760330578512,
  'recall': 0.8917525773195877,
  'f1': 0.793577981651376,
  'number': 194},
 'ORGANIZACAO': {'precision': 0.9028974158183242,
  'recall': 0.8604477611940299,
  'f1': 0.8811616354604509,
  'number': 1340},
 'PESSOA': {'precision': 0.9172597864768683,
  'recall': 0.9617537313432836,
  'f1': 0.9389799635701275,
  'number': 1072},
 'TEMPO': {'precision': 0.965311004784689,
  'recall': 0.9889705882352942,
  'f1': 0.9769975786924939,
  'number': 816},
 'overall_precision': 0.8967979229770662,
 'overall_recall': 0.8913978494623656,
 'overall_f1': 0.8940897325280415,
 'overall_accuracy': 0.9751778993402106}

#Test

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results_test = metric.compute(predictions=true_predictions, references=true_labels)
results_test

{'JURISPRUDENCIA': {'precision': 0.8187976291278577,
  'recall': 0.8986988847583643,
  'f1': 0.8568896765618077,
  'number': 1076},
 'LEGISLACAO': {'precision': 0.9484346224677717,
  'recall': 0.9229390681003584,
  'f1': 0.935513169845595,
  'number': 558},
 'LOCAL': {'precision': 0.5934065934065934,
  'recall': 0.7941176470588235,
  'f1': 0.6792452830188679,
  'number': 68},
 'ORGANIZACAO': {'precision': 0.8600823045267489,
  'recall': 0.875392670157068,
  'f1': 0.8676699532952776,
  'number': 955},
 'PESSOA': {'precision': 0.9028960817717206,
  'recall': 0.9742647058823529,
  'f1': 0.9372236958443856,
  'number': 544},
 'TEMPO': {'precision': 0.9936908517350158,
  'recall': 0.9224011713030746,
  'f1': 0.9567198177676537,
  'number': 683},
 'overall_precision': 0.8812375249500998,
 'overall_recall': 0.9093717816683831,
 'overall_f1': 0.8950836289913836,
 'overall_accuracy': 0.9820312614877881}

# salvando modelo

In [ ]:
model_dir = '/content/drive/MyDrive/' + 'ner-lenerbr-' + str(model_name) + '/model/'
trainer.save_model(model_dir)

# FIM